# Extracting entities given the dataset with help of LLM prompt
Entity Extraction with Generative Models


In [1]:
# importing libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
# reading the data
df = pd.read_json('../data/relations_dev.txt')

In [39]:
#df_nested_list.head()

In [6]:
import json
# load data using Python JSON module
with open('../data/relations_dev.txt','r') as f:
    data = json.loads(f.read())
# Flatten data
df_nested_list2 = pd.json_normalize(data, record_path =['relations'])

In [12]:
#df_nested_list2 = pd.json_normalize('../data/relations_dev.txt', 'r')

In [18]:
#df_nested_list =pd.merge(df_nested_list, df_nested_list, )

concatenated = pd.concat([df_nested_list, df_nested_list2], axis="columns")

In [19]:
concatenated.head()

,text,start,end,token_start,token_end,entityLabel,child,head,relationLabel
0,Bachelor,0,8,0,0,DIPLOMA,4.0,0.0,DEGREE_IN
1,Mechanical Engineering,21,43,4,5,DIPLOMA_MAJOR,7.0,0.0,DEGREE_IN
2,Physical Science,47,63,7,8,DIPLOMA_MAJOR,15.0,9.0,EXPERIENCE_IN
3,3+ years,64,72,9,11,EXPERIENCE,18.0,9.0,EXPERIENCE_IN
4,developing,89,99,15,15,SKILLS,22.0,9.0,EXPERIENCE_IN


In [25]:
#clean_df = pd.concat([df1, concatenated], axis="columns")

In [41]:
#clean_df.head()

In [28]:
clean_df.shape

(145, 10)

In [29]:
#clean_df.to_csv('../data/relation.csv')

In [40]:
#clean_df.head()

In [32]:
# installing the libe
!pip install cohere requests tqdm

In [33]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm
pd.set_option('display.max_colwidth', None)

def get_post_titles():
    """ Gets data from the pushshift api. Read more: https://github.com/pushshift/api """
    #base_url = f"https://api.pushshift.io/reddit/search/submission/"
    #base_data = f"../data/relations_dev.txt"
    #payload = kwargs
    #request = requests.get(base_url, params=payload)
    #return [a['title'] for a in request.json()['data']]
    data = pd.read_json('../data/relations_dev.txt')
    return data['document']




In [42]:
my_API = "nMqcDQ2gC0vxFVwIDvUINps6zJTvYZOskiz5dhq1"

In [43]:
# Paste your API key here. Remember to not share publicly
#api_key = ''

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(my_API)

# Preparing examples for the prompt
In our prompt, we'll present the model with examples for the type of output we're after. We basically get a set of subreddit article titles, and label them ourselves. The label here is the name of the movie mentioned in the title (and "none" if no movie is mentioned).

In [47]:
# job description
movie_examples= [("Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience"),
("10+ years of software engineering work experience. Technical experience in release automation engineering, CI/CD or related roles. Experience building and leading a software organization through product design, delivery and commercialization of consumer electronics devices. Experience recruiting and managing technical teams, including performance management. BS/MS in Computer Science. Experience in leading timeline, multi-partner initiatives. Organizational communication and coordination experience. PREFERRED 5+ years of experience with hands-on technical management, release engineering, tools engineering, DevOps, or related area."),
("3+ years Swift & Objective-C and experience with iOS internals Experience building an entire app from scratch and ideally a portfolio of apps featured in the App Store Someone who knows every trick in the book on UI transitions, network communication and memory/battery efficiency Strong UI/design skill experience is a plus"),
("8+ years experience in software engineering leadership 5+ years people management experience including managing leaders and managing remotely across regions Strategic thinker with proven track record of transforming operations to provide customer experience through innovation and improvement Track record of working with VP, C-level Executives Experience deploying operational support models across enterprise organizations Communication/presentations experience Experience working with all levels of management internally and externally Experience meeting objectives in an entrepreneurial environment Collaboration and relationship-building experience BA/BS degree or higher")
]

## Creating the extraction prompt
We'll create a prompt that demonstrates the task to the model. The prompt contains the examples above, and then presents the input text and asks the model to extract the movie name.

In [51]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereExtractor():
    def __init__(self, examples, example_labels, labels, task_desciption, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_desciption = task_desciption
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_desciption +
                "\n---\n".join( [examples[i] + "\n" +
                                self.example_prompt + 
                                 labels[i] for i in range(len(examples))]))

    def extract(self, example):
        extraction = co.generate(
          model='large',
          prompt=self.make_prompt(example),
          max_tokens=10,
          temperature=0.1,
          stop_sequences=["\n"])
        return(extraction.generations[0].text[:-1])


cohereMovieExtractor = cohereExtractor([e[1] for e in movie_examples], 
                                       [e[0] for e in movie_examples], [],
                                       "", 
                                       "extract the title of the job description:")

# Uncomment to inspect the full prompt:
# print(cohereMovieExtractor.make_prompt('<input text here>'))

In [52]:
# This is what the prompt looks like:
print(cohereMovieExtractor.make_prompt('"EXPERIENCE"'))

a
extract the title of the job description:B
---
0
extract the title of the job description:1
---
+
extract the title of the job description:3
---
+
extract the title of the job description:8
---
"EXPERIENCE"
extract the title of the job description:


## Getting the data
Let's now make the API call to get the top posts for 2021 from r/movies.

In [ ]:
num_posts = 10

movies_list = get_post_titles(size=num_posts, 
      after=str(int(datetime.datetime(2021,1,1,0,0).timestamp())), 
      before=str(int(datetime.datetime(2022,1,1,0,0).timestamp())), 
      subreddit="movies", 
      sort_type="score", 
      sort="desc")

# Show the list
movies_list

## Running the model
And now we loop over the posts and process each one of them with our extractor.

In [ ]:
results = []
for text in tqdm(movies_list):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

In [ ]:
# look at the result
pd.DataFrame(data={'text': movies_list, 'extracted_text': results})

## How well does this work?
We can better measure the performance of this extraction method using a larger labeled dataset. So let's load a test set of 100 examples:

In [ ]:
test_df = pd.read_csv('https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/movie_extraction_test_set_100.csv',index_col=0)
test_df

Let's run the extractor on these post titles (calling the API in parallel for quicker results):

In [ ]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['text']):
        extracted.append(str(i).strip())
# Save results
test_df['extracted_text'] = extracted

In [ ]:
#Let's look at some results:

test_df.head()

Let's calculate the accuracy by comparing to the labeled examples

In [ ]:
# Compare the label to the extracted text
test_df['correct'] = (test_df['label'].str.lower() == test_df['extracted_text'].str.lower()).astype(int)

# Print the accuracy
print(f'Classification accuracy {test_df["correct"].mean() *100}%')

So it seems this prompt works well on this small test set. It's not guaranteed it will do as well on other sets, however. The prompt can be improved by trying on more data, discovering edge cases, and adding more examples to the prompt.

We can look at the examples it got wrong: